In [ ]:
import os

os.environ["HF_TOKEN"]  = "----"

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
system_message = """You are Qwen, an expert vision-language reasoning assistant.
Your goal is to achieve the highest accuracy in visual reasoning tasks.
Follow this rigid process:
1. PERCEPTION: Identify specific objects, colors, shapes, materials, and their exact spatial relations.
2. REASONING: Connect the visual evidence logically to the question.
3. CONCLUSION: Formulate the final answer.

Output format (JSON):
{
  "explanation": "<one short sentence, less than 30 words>",
  "answer": "<one word only>"
}
"""

In [ ]:
import os
import json
import ast
from PIL import Image

# 假设你的图片在这个文件夹
IMAGE_ROOT = "/home/org/JAIST/advanced_machine_learning/data/custom_dataset/custom_dataset/train"

import random

# 修改原代码 [cite: 83-142]
def format_data(sample):
    # 1. 路径处理
    image_file_path = os.path.join(IMAGE_ROOT, sample["file"])

    # 2. Explanation 清洗策略：避免拼接！
    raw_explanation = sample.get("explanation", "")
    explanation_str = ""

    # 解析可能存在的字符串列表
    if isinstance(raw_explanation, str) and raw_explanation.strip().startswith("["):
        try:
            exp_list = ast.literal_eval(raw_explanation)
        except:
            exp_list = [str(raw_explanation)]
    elif isinstance(raw_explanation, list):
        exp_list = raw_explanation
    else:
        exp_list = [str(raw_explanation)]

    # [关键修改]：不要 join！改为选择最短的一条作为本次训练的目标。
    if isinstance(exp_list, list) and len(exp_list) > 0:
        # 选最短的一条
        explanation_str = max(exp_list, key=lambda x: len(str(x)))
    else:
        explanation_str = str(raw_explanation)

    # 清洗
    explanation_str = clean_text(str(explanation_str).strip())
    # 3. 构造 JSON (保持 Explanation 在前以利用 CoT 提升 Answer 准确率)
    answer_json = {
        "explanation": explanation_str,
        "answer": str(sample["answer"]).strip().lower() # 强制训练目标为小写，匹配评分逻辑
    }

    return {
        "images": [image_file_path],
        "messages": [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_message}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image_file_path},
                    {"type": "text", "text": sample["question"]}
                ]
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": json.dumps(answer_json, ensure_ascii=False)}]
            }
        ]
    }

# 辅助函数：复用评分脚本里的 clean_text，让训练目标更纯净
def clean_text(text):
    punctuations = '''!()-[]{};:'"\.,<>/?@#$%^&*_~'''
    for p in punctuations:
        text = text.replace(p, "")
    text = text.lower()
    return text

<>:69: SyntaxWarning: invalid escape sequence '\.'
<>:69: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_990505/2739096182.py:69: SyntaxWarning: invalid escape sequence '\.'
  punctuations = '''!()-[]{};:'"\.,<>/?@#$%^&*_~'''


For educational purposes, we’ll load only 10% of each split in the dataset. However, in a real-world use case, you would typically load the entire set of samples.


In [22]:
from datasets import load_dataset

# 1️⃣ 加载整个 CSV 文件
dataset_dict = load_dataset("csv", data_files="/home/org/JAIST/advanced_machine_learning/data/custom_dataset/custom_dataset/train_labels.csv")

# 取出真正的数据
dataset = dataset_dict["train"]

# 2️⃣ 先划分 test (10%)
split_1 = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_1["train"]
test_dataset  = split_1["test"]

# 3️⃣ 从 train 中再划分 val (10%)
split_2 = train_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_2["train"]
val_dataset   = split_2["test"]

# 4️⃣ 打印
print("Train:", len(train_dataset))
print("Val:", len(val_dataset))
print("Test:", len(test_dataset))


Train: 4050
Val: 450
Test: 500


Let’s take a look at the structure of the dataset. It includes an image, a query, a label (which is the answer), and a fourth feature that we’ll be discarding.


In [23]:
train_dataset

Dataset({
    features: ['id', 'file', 'question', 'answer', 'explanation'],
    num_rows: 4050
})

Now, let’s format the data using the chatbot structure. This will allow us to set up the interactions appropriately for our model.


In [24]:
train_dataset = [format_data(sample) for sample in train_dataset]
eval_dataset = [format_data(sample) for sample in val_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]

In [25]:
train_dataset[1]

{'images': ['/home/org/zc/data/custom_dataset/custom_dataset/train/225510d7b4eb0d447441053778119488.png'],
 'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are Qwen, an expert vision-language reasoning assistant.\nYour goal is to achieve the highest accuracy in visual reasoning tasks.\nFollow this rigid process:\n1. PERCEPTION: Identify specific objects, colors, shapes, materials, and their exact spatial relations.\n2. REASONING: Connect the visual evidence logically to the question.\n3. CONCLUSION: Formulate the final answer.\n\nOutput format (JSON):\n{\n  "explanation": "<concise reasoning sentence containing key visual attributes and spatial keywords>",\n  "answer": "<single word or number, lowercase>"\n}\n'}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': '/home/org/zc/data/custom_dataset/custom_dataset/train/225510d7b4eb0d447441053778119488.png'},
    {'type': 'text',
     'text': 'What material is the tiny block that is behin

In [26]:
import torch
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

model_id = "Qwen/Qwen3-VL-2B-Instruct"

Next, we’ll load the model and the tokenizer to prepare for inference.

In [27]:
import gc
import time

def clear_memory():
    # Delete variables if they exist in the current global scope
    if 'inputs' in globals(): del globals()['inputs']
    if 'model' in globals(): del globals()['model']
    if 'processor' in globals(): del globals()['processor']
    if 'trainer' in globals(): del globals()['trainer']
    if 'bnb_config' in globals(): del globals()['bnb_config']
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

clear_memory()

GPU allocated memory: 0.10 GB
GPU reserved memory: 0.12 GB


## 4. Fine-Tune the Model using TRL


### 4.1 Load the Quantized Model for Training ⚙️

Next, we’ll load the quantized model using [bitsandbytes](https://huggingface.co/docs/bitsandbytes/main/en/index). If you want to learn more about quantization, check out [this blog post](https://huggingface.co/blog/merve/quantization) or [this one](https://www.maartengrootendorst.com/blog/quantization/).


In [28]:
# Load model and tokenizer
model = Qwen3VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

# # ==== 冻结视觉编码器 ====
# for param in model.visual.parameters():
#     param.requires_grad = False
    
# print("✅ Visual encoder has been frozen.")

processor = AutoProcessor.from_pretrained(model_id)

In [29]:
next(model.parameters()).device

device(type='cuda', index=0)

### 4.2 Set Up QLoRA and SFTConfig 🚀

Next, we will configure [QLoRA](https://github.com/artidoro/qlora) for our training setup. QLoRA enables efficient fine-tuning of large language models while significantly reducing the memory footprint compared to traditional methods. Unlike standard LoRA, which reduces memory usage by applying a low-rank approximation, QLoRA takes it a step further by quantizing the weights of the LoRA adapters. This leads to even lower memory requirements and improved training efficiency, making it an excellent choice for optimizing our model's performance without sacrificing quality.




In [30]:

from peft import LoraConfig

peft_config = LoraConfig(
    r=64,             
    lora_alpha=128,   
    lora_dropout=0.05,
    bias="none",
    # ✅ 核心修改：覆盖 Attention 和 MLP (Feed Forward) 所有层
    target_modules=[
        "q_proj", "v_proj", "k_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ], 
    task_type="CAUSAL_LM",
    # modules_to_save=None # 如果只有文字没问题但格式总不对，可以考虑把 ["lm_head"] 加入这里，但会显著增加显存
)

In [31]:
from trl import SFTConfig
# 修改原代码 [cite: 242-271]
training_args = SFTConfig(
    output_dir="qwen3-vl-2b-instruct-trl-sft-CLEVR-Optimized",
    num_train_epochs=1,
    per_device_train_batch_size=8, # 如果显存允许，尝试改为 8
    gradient_accumulation_steps=2, # 如果 Batch 增大，这里可以适当减小

    # 优化器设置
    learning_rate=1e-4, # Qwen-2B 对稍大的 LR 耐受度尚可，LoRA 可以稍微激进一点
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    optim="adamw_torch_fused",
    neftune_noise_alpha=5,

    # 其他设置
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    bf16=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    


    logging_steps=10,
    save_strategy="epoch", # 按 epoch 保存，防止存太多 checkpoint 爆硬盘
    eval_strategy="no",    # 为了节省时间，可以关掉 eval，或者只在最后 eval
    report_to="trackio",


)

### 4.3 Training the Model 🏃

We will log our training progress using [trackio](https://huggingface.co/blog/trackio). Let’s connect our notebook to W&B to capture essential information during training.


In [32]:
import trackio

trackio.init(
    project="qwen3-vl-2b-instruct-trl-sft-CLEVR",
    name="qwen3-vl-2b-instruct-trl-sft-CLEVR",
    config=training_args,
    space_id=training_args.output_dir + "-CLEVR"
)

* Trackio project initialized: qwen3-vl-2b-instruct-trl-sft-CLEVR
* Trackio metrics will be synced to Hugging Face Dataset: Org-Huang/qwen3-vl-2b-instruct-trl-sft-CLEVR-Optimized-CLEVR-dataset
* Found existing space: https://huggingface.co/spaces/Org-Huang/qwen3-vl-2b-instruct-trl-sft-CLEVR-Optimized-CLEVR
* View dashboard by going to: https://Org-Huang-qwen3-vl-2b-instruct-trl-sft-CLEVR-Optimized-CLEVR.hf.space/


* Created new run: qwen3-vl-2b-instruct-trl-sft-CLEVR


Now, we will define the [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer), which is a wrapper around the [transformers.Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) class and inherits its attributes and methods. This class simplifies the fine-tuning process by properly initializing the [PeftModel](https://huggingface.co/docs/peft/v0.6.0/package_reference/peft_model) when a [PeftConfig](https://huggingface.co/docs/peft/v0.6.0/en/package_reference/config#peft.PeftConfig) object is provided. By using `SFTTrainer`, we can efficiently manage the training workflow and ensure a smooth fine-tuning experience for our Vision Language Model. When doing inference we defined our own `generate_text_from_sample` function which applied the necessary preprocessing before passing the inputs to the model. Here, the SFTTrainer infers automatically that the model is a vision-language model and applies a `DataCollatorForVisionLanguageModeling` which convers the inputs to the appropriate format.



In [33]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    processing_class=processor,

)

The model is already on multiple devices. Skipping the move to device specified in `args`.


Time to Train the Model! 🎉

In [34]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151645, 'bos_token_id': None, 'pad_token_id': 151643}.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: Org-Huang/trackio-dataset
* Found existing space: https://huggingface.co/spaces/Org-Huang/trackio
* View dashboard by going to: https://Org-Huang-trackio.hf.space/


* Created new run: Org-Huang-1764661993


Step,Training Loss
10,11.413000
20,5.126100
30,3.788500
40,3.657800
50,3.624200
60,3.628900
70,3.640900
80,3.620000
90,3.596900
100,3.604600


* Run finished. Uploading logs to Trackio (please wait...)


TrainOutput(global_step=254, training_loss=3.9819463519599494, metrics={'train_runtime': 307.1125, 'train_samples_per_second': 13.187, 'train_steps_per_second': 0.827, 'total_flos': 1.602763502690304e+16, 'train_loss': 3.9819463519599494, 'entropy': 3.617557866232736, 'num_tokens': 1335590.0, 'mean_token_accuracy': 0.5219489591462272, 'epoch': 1.0})

Let's save the results 💾

In [35]:
trainer.save_model(training_args.output_dir)

## 5. Testing the Fine-Tuned Model 🔍

Now that we've successfully fine-tuned our Vision Language Model (VLM), it's time to evaluate its performance! In this section, we will test the model using examples from the ChartQA dataset to see how well it answers questions based on chart images. Let's dive in and explore the results! 🚀



Let's clean up the GPU memory to ensure optimal performance 🧹

In [36]:
clear_memory()

GPU allocated memory: 0.10 GB
GPU reserved memory: 0.12 GB
